In [1]:
import sys
import numpy as np
from black_scholes import normalised_price, normalised_implied_volatility
import numpy as np
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import ipywidgets as widgets
import datetime

init_notebook_mode(connected=True)

In [2]:
sys.path

data = pd.read_csv("C:\\Users\\Samuel\\Documents\\python\\data\\quotedata.csv",
           skiprows=2)

calls = data[data.columns[:data.shape[1]/2]].copy()
calls.columns = ["Description"] + list(calls.columns[1:])

puts = data[data.columns[data.shape[1]/2:]].copy()
puts.columns = calls.columns

calls["Type"] = "C"
puts["Type"] = "P"

data = pd.concat([calls,puts], axis=0) # discard the old "data"

split_description = pd.DataFrame(data["Description"].str.split(" ").tolist())

expirations = pd.to_datetime(split_description[0]+" "+split_description[1]+" "+split_description[2])
strikes = split_description[3]
code = split_description[4]

data["Expiration"] = expirations
data["Strike"] = strikes
data["Code"] = code.str.replace("(","").str.replace(")","")

data["Weekly"] = data["Code"].str[:4]=="SPXW"

In [3]:
split = pd.DataFrame((data["Code"] + "-").str.split("-").tolist())

In [4]:
date_and_strike = split[0].str.replace("SPXW","").str.replace("SPX","")
data["Exchange"] = split[1]
data["Year"] = date_and_strike.str[:2].astype(int)
data["Day"] = date_and_strike.str[2:4].astype(int)
data["Month"] = date_and_strike.str[4].astype(str)
data["Strike"] = date_and_strike.str[5:].astype(float)

In [5]:
# For each option, it seems we have data for exchange code "E" corresponding to CBOE, and "" (blank) corresponding to "Composite"
# see http://www.cboe.com/delayedquote/help#chaindata
# i.e. everything is duplicated (i did spot-check that the numbers are identical), let's just delete the dupes
exchange_code_count = data.groupby("Exchange").size()
assert len(exchange_code_count)==2
assert len(exchange_code_count.unique())==1
data = data[0==data["Exchange"].str.len()]
del data["Exchange"]

# tidy up the column names a bit
data.columns = [{"vol":"volume","last_sale":"last"}.get(x,x) for x in [x.lower().replace(" ","_") for x in data.columns]]

In [6]:
from clean_option_data import preprocess_option_data, preprocess_option_expiry

as_of = datetime.date(2018,8,14)

data = data.groupby(["expiration", "type", "strike"]).first().reset_index() # hmmm... what are we losing here?
#processed = preprocess_option_data(data, as_of)



In [7]:
d = preprocess_option_data(data, as_of)

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(284, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(550, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(282, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(280, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(408, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\core\numeric.py:301: FutureWarning:

in the future, full(394, False) will return an array of dtype('bool')

C:\Users\Samuel\Anaconda2\lib\site-packages\numpy\co

In [8]:
d.sample()

,,,expiration,type,strike,description,last,net,bid,ask,volume,open_int,...,day,month,mid,outlier,forward,disc_fac,expiration_time,mid_vol,bid_vol,ask_vol
expiration,type,strike,,,,,,,,,,,,,,,,,,,,,
2018-12-31,P,2750.0,2018-12-31,P,2750.0,2018 Dec 31 2750.00 (SPXW1831X2750),67.45,11.37,62.9,63.9,5,1128,...,31,L,63.4,0,2828.360882,0.990756,0.380822,0.143123,0.142349,0.143896


In [9]:

grouped_data = list(d.groupby('expiration'))

data = [go.Scatter(x=x[1]["strike"].values, y=x[1]["mid_vol"].values, name=str(x[0]), mode = 'markers') for x in grouped_data]
layout = go.Layout(title='Volatility', yaxis = {'title': 'Volatiltiy'}, xaxis = {'title' : 'Strike'})
figure = go.Figure(data=data, layout=layout)
iplot(figure)

In [10]:
def update_plot(vol_atm, vol_vol, corr, alpha, beta):
    ''' Update the plot with the newly set slider values'''

    alpha = -np.log((1-alpha))
    beta = -np.log((1-beta))
    x = np.linspace(-2,2,1001)
    fx = np.sinh(beta*x)-beta*x + np.tanh(alpha*x)/alpha
    f1x = 1
    f2x = 0

    vol = sabr_volatility(x, x, 1.0, 0.0, vol_atm, vol_vol, corr, compute_derivatives = False)
    vol_2 = sabr_volatility(x, fx, np.nan, np.nan, vol_atm, vol_vol, corr, compute_derivatives = False)

    data = [go.Scatter(x=x, y=vol, mode = 'lines', name='3-param'),
            go.Scatter(x=x, y=vol_2, mode = 'lines', name='5-param')
           ]
    layout = go.Layout(title='SABR vol curve', yaxis = {'title': 'Volatility'}, xaxis = {'title' : 'ln(K/F)'})
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)
    
data = widgets.Select(options = sorted(test.keys()), value = min(list(test.keys())))
vol_atm_slider = widgets.FloatSlider(min=0, max=3, value = 0.1, description='v0')
vol_vol_slider = widgets.FloatSlider(min=0, max=3, value = 0.3, description='vv')
corr_slider = widgets.FloatSlider(min=-1, max=1, value = -0.7, description='c')
alpha_slider = widgets.FloatSlider(min=0.0, max=1.0, value = 0.1, description='a')
beta_slider = widgets.FloatSlider(min=0.0, max=1.0, value = 0.1, description='b')

controls = widgets.VBox([widgets.HBox([vol_atm_slider, vol_vol_slider, corr_slider]),
    widgets.HBox([alpha_slider, beta_slider])])

update_plot(vol_atm_slider.value, vol_vol_slider.value, corr_slider.value, alpha_slider.value, beta_slider.value)
widgets.interactive(update_plot, vol_atm = vol_atm_slider, vol_vol = vol_vol_slider, corr = corr_slider, alpha = alpha_slider, beta=beta_slider)
controls

NameError: name 'test' is not defined

In [ ]:
df = grouped_data[4][1]

df.columns


In [ ]:
test = dict(grouped_data)
test = {a.date():b for a, b in test.iteritems()}

def update_plot(vol_atm, vol_vol, corr, alpha, beta, maturity):
    ''' Update the plot with the newly set slider values'''

    maturity_data = test[maturity]
    print(len(maturity_data.columns))
    
    calls = maturity_data[maturity_data["type"]=="C"]
    puts = maturity_data[maturity_data["type"]=="P"]
    
    fwd = maturity_data["forward"].unique()[0]
    k = np.linspace(min(maturity_data["strike"]), max(maturity_data["strike"]), 1001)
    x = np.log(k/fwd)
    
    alpha = -np.log((1-alpha))
    beta = -np.log((1-beta))
    fx = np.sinh(beta*x)-beta*x + np.tanh(alpha*x)/alpha
    f1x = 1
    f2x = 0

    vol = sabr_volatility(x, fx, np.nan, np.nan, vol_atm, vol_vol, corr, compute_derivatives = False)

    data = [go.Scatter(x=k, y=vol, mode = 'lines', name='5-param'),
            go.Scatter(x=calls["strike"], y=calls["bid_vol"], mode = 'markers', name='call bid'),
            go.Scatter(x=calls["strike"], y=calls["ask_vol"], mode = 'markers', name='call ask'),
            go.Scatter(x=puts["strike"], y=puts["bid_vol"], mode = 'markers', name='put bid'),
            go.Scatter(x=puts["strike"], y=puts["ask_vol"], mode = 'markers', name='put ask')
           ]
    layout = go.Layout(title='SABR vol curve', yaxis = {'title': 'Volatility'}, xaxis = {'title' : 'ln(K/F)'})
    figure = go.Figure(data=data, layout=layout)
    iplot(figure)
    
data = widgets.Select(options = sorted(test.keys()), value = min(list(test.keys())))
vol_atm_slider = widgets.FloatSlider(min=0, max=3, value = 0.1, description='v0', orientation = 'vertical')
vol_vol_slider = widgets.FloatSlider(min=0, max=3, value = 0.3, description='vv', orientation = 'vertical')
corr_slider = widgets.FloatSlider(min=-1, max=1, value = -0.7, description='c', orientation = 'vertical')
alpha_slider = widgets.FloatSlider(min=0.0, max=1.0, value = 0.1, description='a', orientation = 'vertical')
beta_slider = widgets.FloatSlider(min=0.0, max=1.0, value = 0.1, description='b', orientation = 'vertical')
maturity_select = widgets.Dropdown(options=sorted(test.keys()))

x_scale = widgets.RadioButtons(
    options=['strike', 'moneyness', 'log-moneyness', 'delta'],
#     value='pineapple',
    description='X-axis:',
    disabled=False
)

y_scale = widgets.RadioButtons(
    options=['volatility', 'volatility error', 'price', 'price error'],
#     value='pineapple',
    description='Y-axis:',
    disabled=False
)

display = widgets.RadioButtons(
    options=["bid/ask", "mid"],
    description="Display",
    disabled=False
)

button = widgets.Button(description = "Fit")

def on_button_clicked(b):
    vol_vol_slider.value = 0.7
    
button.on_click(on_button_clicked)

axis_controls = widgets.VBox([x_scale, y_scale, display])
other_controls = widgets.VBox([maturity_select, button])

controls = widgets.HBox([other_controls, vol_atm_slider, vol_vol_slider, corr_slider, alpha_slider, beta_slider, axis_controls])

update_plot(vol_atm_slider.value, vol_vol_slider.value, corr_slider.value, alpha_slider.value, beta_slider.value, maturity_select.value)
widgets.interactive(update_plot, vol_atm = vol_atm_slider, vol_vol = vol_vol_slider, corr = corr_slider, alpha = alpha_slider, beta=beta_slider, maturity=maturity_select)
controls

In [ ]:
a = 0.3
b = 0.1
f = np.sinh(a*x) - a*x + np.tanh(b*x)/b

#f = np.tanh(b*x)/b


data = []
data.append(go.Scatter(x=x, y=x, mode = 'lines', name = 'x'))
data.append(go.Scatter(x=x, y=f, mode = 'lines', name='f'))
layout = go.Layout(title='SABR vol curve', yaxis = {'title': 'Volatility'}, xaxis = {'title' : 'ln(K/F)'})
figure = go.Figure(data=data, layout=layout)
iplot(figure)
